In [ ]:
import import_ipynb
from Json2Graph import json_to_graph
from DrawGraph import draw_nodes, draw_edges, draw_labels, draw_graph
import pandas as pd
import numpy as np
import networkx as nx
from networkx.algorithms import community
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
G = json_to_graph('data/dataset_20180403/180402_Lukas_Herkunfts-_Pflegefamilie_Schule.json')

In [ ]:
print('Weighted Degree:')
print(G.degree(weight='weight'))
print('Positive Weighted Degree:')
print(G.degree(weight='strengthen'))
print('Negative Weighted Degree:')
print(G.degree(weight='weaken'))
print('Absolute Weighted Degree:')
print(G.degree(weight='weight_absolute'))
print('Degree:')
print(G.degree())
print('Weighted In-Degree:')
print(G.in_degree(weight='weight'))
print('Positive Weighted In-Degree:')
print(G.in_degree(weight='strengthen'))
print('Negative Weighted In-Degree:')
print(G.in_degree(weight='weaken'))
print('Absolute Weighted Degree:')
print(G.in_degree(weight='weight_absolute'))
print('In-Degree:')
print(G.in_degree())
print('Weighted Out-Degree:')
print(G.out_degree(weight='weight'))
print('Positive Weighted Out-Degree:')
print(G.out_degree(weight='strengthen'))
print('Negative Weighted Out-Degree:')
print(G.out_degree(weight='weaken'))
print('Absolute Weighted Degree:')
print(G.out_degree(weight='weight_absolute'))
print('Out-Degree:')
print(G.out_degree(G))

In [ ]:
w = -2
print(abs(w) if np.sign(w) == 1 else 0)
print(abs(w) if np.sign(w) == -1 else 0)

In [ ]:
for n, nbrsdict in G.adjacency():
    for nbr, keydict in nbrsdict.items():
        for key, eattr in keydict.items():
            print(eattr)
            pass

In [ ]:
from networkx.algorithms import approximation as ap
print('Approximation:')
print('all_pairs_node_connectivity:')
apc = ap.all_pairs_node_connectivity(G)
for key, value in apc.items():
    print('{}: {}'.format(key, value))
print('node_connectivity: {}'.format(ap.node_connectivity(G)))
#ap.k_components(G) - only undirected
print('max_clique: {}'.format(ap.max_clique(G)))
#ap.average_clustering(G) - only undirected
#ap.min_weighted_dominating_set(G, weight='weight') - only undirected
print('maximum_independent_set: {}'.format(ap.maximum_independent_set(G)))
print('min_maximal_matching: {}'.format(ap.min_maximal_matching(G)))
print('ramsey_R2: {}'.format(ap.ramsey_R2(G)))
#ap.metric_closure(G, weight='weight') - only undirected
print('min_weighted_vertex_cover: {}'.format(ap.min_weighted_vertex_cover(G, weight='weight')))

In [ ]:
from networkx.algorithms import community as co
import itertools

k = 20
comp = co.girvan_newman(G)
for communities in itertools.islice(comp, k):
    print(tuple(sorted(c) for c in communities)) 

In [ ]:
def add_to_dataframe(df, name, func, weights=[]):
    if len(weights) == 0:
        data = func(G)
        kwargs = {name : list(data.values())}
        df = df.assign(**kwargs)
    else:
        for i, weight in enumerate(weights):
            data = func(G, weight=weight)
            kwargs = {'{} - {}'.format(name, weight) : list(data.values())}
            df = df.assign(**kwargs)
    return df
    
df = pd.DataFrame()
df = add_to_dataframe(df, 'degree_centrality', nx.degree_centrality)
df = add_to_dataframe(df, 'in_degree_centrality', nx.in_degree_centrality)
df = add_to_dataframe(df, 'out_degree_centrality', nx.out_degree_centrality)
df = add_to_dataframe(df, 'eigenvector_centrality_numpy', nx.eigenvector_centrality_numpy)
df = add_to_dataframe(df, 'eigenvector_centrality_numpy', nx.eigenvector_centrality_numpy, ['weight','weight_absolute','strengthen','weaken'])
df = add_to_dataframe(df, 'closeness_centrality', nx.degree_centrality)
df = add_to_dataframe(df, 'betweenness_centrality', nx.betweenness_centrality)
df = add_to_dataframe(df, 'betweenness_centrality', nx.betweenness_centrality, ['weight','weight_absolute','strengthen','weaken'])
#df = add_to_dataframe(df, 'edge_betweenness_centrality', nx.edge_betweenness_centrality)
#df = add_to_dataframe(df, 'edge_betweenness_centrality', nx.edge_betweenness_centrality, ['weight','weight_absolute','strengthen','weaken'])
#df = add_to_dataframe(df, 'approximate_current_flow_betweenness_centrality', nx.approximate_current_flow_betweenness_centrality)
#df = add_to_dataframe(df, 'approximate_current_flow_betweenness_centrality', nx.approximate_current_flow_betweenness_centrality, ['weight','weight_absolute','strengthen','weaken'])
df = add_to_dataframe(df, 'load_centrality', nx.load_centrality)
df = add_to_dataframe(df, 'load_centrality', nx.load_centrality, ['weight_absolute','strengthen','weaken'])
#df = add_to_dataframe(df, 'edge_load_centrality', nx.edge_load_centrality)
df = add_to_dataframe(df, 'harmonic_centrality', nx.harmonic_centrality)
df = add_to_dataframe(df, 'betweenness_centrality', nx.betweenness_centrality, ['weight','weight_absolute','strengthen','weaken'])


print('edge_betweenness_centrality: {}'.format(nx.edge_betweenness_centrality(G)))
print('edge_betweenness_centrality (weighted): {}'.format(nx.edge_betweenness_centrality(G, weight='weight')))
print('approximate_current_flow_betweenness_centrality: {}'.format(nx.edge_betweenness_centrality(G)))
print('approximate_current_flow_betweenness_centrality (weighted): {}'.format(nx.edge_betweenness_centrality(G, weight='weight')))
print('edge_load_centrality: {}'.format(nx.load_centrality(G)))
print('global_reaching_centrality: {}'.format(nx.global_reaching_centrality(G)))

In [ ]:
df

In [ ]:
def amplify_network(G, alpha=1.):
    local_g = G.copy()
    egde_traversal = list(nx.edge_dfs(local_g))
    influence = nx.get_node_attributes(local_g, 'influence')
    weight_absolute = nx.get_edge_attributes(local_g, 'weight_absolute')
    
    for i, (from_node, to_node, node_key) in enumerate(egde_traversal):
#         print('traveling from {} to {}'.format(from_node, to_node))
#         print('influence of {} before: {}'.format(to_node, influence[to_node]))
#         print('influence[from_node]: {}'.format(influence[from_node]))
#         print('influence[to_node]: {}'.format(influence[to_node]))
        computed_influence = influence[to_node] + (influence[from_node] * (alpha * weight_absolute[(from_node, to_node, 0)]))
#         print('computed_influence: {}'.format(computed_influence))
        if influence[to_node] == 0:
            influence[to_node] = 0
        elif np.sign(influence[to_node]) == 1:
            influence[to_node] = computed_influence if computed_influence > 0 else 0
        else:
            influence[to_node] = computed_influence if computed_influence < 0 else 0
        print('influence of {} after: {}'.format(to_node, influence[to_node]))
#         print()
        
    nx.set_node_attributes(local_g, influence,'influence')
    
    return local_g

In [ ]:
daG = amplify_network(G, alpha=0.05)
print()
for i in range(10):
    daG = amplify_network(daG, alpha=0.05)
    print()

In [ ]:
import json
from collections import defaultdict

def graph_to_json(G):
    json_object = {}
    nodes = []
    links = []
    
    cycles = defaultdict(list)
    
    identifier = nx.get_node_attributes(G, name='id')
    labels = nx.get_node_attributes(G, name='label')
    action_systems_id = nx.get_node_attributes(G, name='actionSystemId')
    action_systems = nx.get_node_attributes(G, name='actionSystem')
    influences = nx.get_node_attributes(G, name='influence')
    
    for i, cycle in enumerate(list(nx.simple_cycles(G))):
        for j, node in enumerate(cycle):
            cycles[node].append(i)

    for i in range(len(identifier)):
        nodes.append({
            'id': identifier[i],
            'label': labels[i],
            'influence': influences[i],
            'actionSystemId': action_systems_id[i],
            'actionSystem': action_systems[i],
            'cycles': cycles[i]
        })

    weight_absolute = nx.get_edge_attributes(G, name='weight_absolute')
    strengthen = nx.get_edge_attributes(G, name='strengthen')
    weaken = nx.get_edge_attributes(G, name='weaken')
    sign = nx.get_edge_attributes(G, name='sign')
    
    
    for (from_node, to_node, weight) in G.edges(data='weight'):
        links.append({
            'source': nodes[from_node]['id'],
            'target': nodes[to_node]['id'],
            'weight': weight,
            'weight_absolute': weight_absolute[(from_node, to_node, 0)],
            'strengthen': strengthen[(from_node, to_node, 0)],
            'weaken': weaken[(from_node, to_node, 0)],
            'sign': sign[(from_node, to_node, 0)]
        })
    
    json_object['nodes'] = nodes
    json_object['links'] = links
    
    return json.JSONEncoder(ensure_ascii=False, allow_nan=False).encode(json_object)

In [ ]:
import codecs

file = codecs.open("json-test.json", "w", "utf-8-sig")
file.write(graph_to_json(G))
file.close()

In [ ]:
graph_to_json(G)

In [ ]:
nx.get_edge_attributes(G, name='weight')

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
pos = nx.circular_layout(G, scale=1)
nx.draw(G, pos, font_size=12, with_labels=False, ax=ax)
edgelabels = nx.get_edge_attributes(G, 'weight')
edgelabels_2d = {}

for (n1, n2, n3), label in edgelabels.items():
    edgelabels_2d[(n1, n2)] = label
nx.draw_networkx_edge_labels(G, pos, edge_labels=edgelabels_2d)
nodelabels = nx.draw_networkx_labels(G, pos, alpha=.8)
